In [110]:
import Pkg; Pkg.activate("..")

  Activating project at `~/SAFT_ML`


In [111]:
using CSV, DataFrames, DelimitedFiles, Clapeyron, PyCall
import PyPlot; const plt = PyPlot ;

In [112]:
# Open and structure Esper et al. Statistics data
# Set missing values to -1 for easier processing (all valid values are positive)

raw_data_statistics = CSV.read("Esper et al Statistics (CSV).csv",DataFrame,header=1) 
processed_data_statistics = raw_data_statistics[1:1842,1:15]

for i = 1:nrow(processed_data_statistics)
    processed_data_statistics[i,:p_sat_AAD_outliers] = replace(processed_data_statistics[i,:p_sat_AAD_outliers], r"[()]" => "")
    processed_data_statistics[i,:rho_vap_AAD_outliers] = replace(processed_data_statistics[i,:rho_vap_AAD_outliers], r"[()]" => "")
    processed_data_statistics[i,:p_sat_n_points_outliers] = replace(processed_data_statistics[i,:p_sat_n_points_outliers], r"[()]" => "")
    processed_data_statistics[i,:rho_vap_n_points_outliers] = replace(processed_data_statistics[i,:rho_vap_n_points_outliers], r"[()]" => "")

    processed_data_statistics[i,:p_sat_AAD_outliers] = replace(processed_data_statistics[i,:p_sat_AAD_outliers], r"-" => -1)
    processed_data_statistics[i,:rho_liq_sat_AAD] = replace(processed_data_statistics[i,:rho_liq_sat_AAD], r"-" => -1)
    processed_data_statistics[i,:rho_liq_sp_AAD] = replace(processed_data_statistics[i,:rho_liq_sp_AAD], r"-" => -1)
    processed_data_statistics[i,:rho_vap_AAD_outliers] = replace(processed_data_statistics[i,:rho_vap_AAD_outliers], r"-" => -1)
    processed_data_statistics[i,:p_sat_n_points_outliers] = replace(processed_data_statistics[i,:p_sat_n_points_outliers], r"-" => -1)
    processed_data_statistics[i,:rho_liq_sat_n_points] = replace(processed_data_statistics[i,:rho_liq_sat_n_points], r"-" => -1)
    processed_data_statistics[i,:rho_liq_sp_n_points] = replace(processed_data_statistics[i,:rho_liq_sp_n_points], r"-" => -1)
    processed_data_statistics[i,:rho_vap_n_points_outliers] = replace(processed_data_statistics[i,:rho_vap_n_points_outliers], r"-" => -1)

end

processed_data_statistics.Name = String.(processed_data_statistics.Name)
processed_data_statistics.CAS = String15.(processed_data_statistics.CAS)
processed_data_statistics.molarweight = Float64.(processed_data_statistics.molarweight)
processed_data_statistics.p_sat_AAD = Float64.(processed_data_statistics.p_sat_AAD)
processed_data_statistics.rho_liq_AAD = Float64.(processed_data_statistics.rho_liq_AAD)

processed_data_statistics.p_sat_AAD_outliers = parse.(Float64, processed_data_statistics.p_sat_AAD_outliers)
processed_data_statistics.rho_liq_sat_AAD = parse.(Float64, processed_data_statistics.rho_liq_sat_AAD)
processed_data_statistics.rho_liq_sp_AAD = parse.(Float64, processed_data_statistics.rho_liq_sp_AAD)
processed_data_statistics.rho_vap_AAD_outliers = parse.(Float64, processed_data_statistics.rho_vap_AAD_outliers)

processed_data_statistics.p_sat_n_points = Int64.(processed_data_statistics.p_sat_n_points)
processed_data_statistics.rho_liq_n_points = Int64.(processed_data_statistics.rho_liq_n_points)

processed_data_statistics.p_sat_n_points_outliers = parse.(Int64, processed_data_statistics.p_sat_n_points_outliers)
processed_data_statistics.rho_liq_sat_n_points = parse.(Int64, processed_data_statistics.rho_liq_sat_n_points)
processed_data_statistics.rho_liq_sp_n_points = parse.(Int64, processed_data_statistics.rho_liq_sp_n_points)
processed_data_statistics.rho_vap_n_points_outliers = parse.(Int64, processed_data_statistics.rho_vap_n_points_outliers)

processed_data_statistics ;

┌ Warning: thread = 1 warning: only found 5 / 17 columns around data row: 1845. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/OnldF/src/file.jl:577


In [113]:
# Open and structure Esper et al. SAFT parameter data
raw_data_parameters = CSV.read("SI_pcp-saft_parameters.csv",DataFrame,header=1) ;
fieldnames(typeof(raw_data_parameters))
#display(names(df)[1:20]) ;
writedlm("out.txt", names(raw_data_parameters), ' ') ;

replace!(raw_data_parameters.mu, missing => 0) ;
replace!(raw_data_parameters.kappa_ab, missing => 0) ;
replace!(raw_data_parameters.epsilon_k_ab, missing => 0) ;

raw_data_parameters.mu = Float64.(raw_data_parameters.mu)
raw_data_parameters.kappa_ab = Float64.(raw_data_parameters.kappa_ab)
raw_data_parameters.epsilon_k_ab = Float64.(raw_data_parameters.epsilon_k_ab) ;
raw_data_parameters = sort(raw_data_parameters, :molarweight) ;

In [114]:
num_rows = nrow(raw_data)
processed_data = DataFrame(
    species = fill(missing, num_rows),
    CAS = fill(missing,num_rows),
    isomeric_SMILES = fill(missing, num_rows), 
    family = fill(missing, num_rows),
    Mw = fill(missing, num_rows),
    segment = fill(missing, num_rows),
    sigma = fill(missing, num_rows),
    epsilon = fill(missing, num_rows),
    dipole = fill(missing, num_rows),
    kappa_ab = fill(missing, num_rows),
    epsilon_k_ab = fill(missing, num_rows),
    na = fill(missing, num_rows),
    nb = fill(missing, num_rows),
    interaction = fill(missing,num_rows),
    bounds_violation = fill(missing,num_rows),
    source = fill(missing, num_rows)
) ;

In [115]:
processed_data.species = raw_data_parameters.common_name
processed_data.CAS = raw_data_parameters.cas
processed_data.Mw = raw_data_parameters.molarweight
processed_data.isomeric_SMILES = raw_data_parameters.isomeric_smiles
processed_data.family = raw_data_parameters.family
processed_data.segment = raw_data_parameters.m
processed_data.sigma = raw_data_parameters.sigma
processed_data.epsilon = raw_data_parameters.epsilon_k
processed_data.dipole = raw_data_parameters.mu
processed_data.kappa_ab = raw_data_parameters.kappa_ab
processed_data.epsilon_k_ab = raw_data_parameters.epsilon_k_ab
processed_data.na = raw_data_parameters.na
processed_data.nb = raw_data_parameters.nb
processed_data.interaction = raw_data_parameters.opt
processed_data.bounds_violation = raw_data_parameters.bounds_violation
processed_data[!, :source] .= "10.1021/acs.iecr.3c02255" ;

species_names = processed_data.species ;
# processed_data = processed_data[1:1843,:] ;
filter!(row -> row.species != "cis-2-butene", processed_data)
filter!(row -> row.species != "(cis/trans)-2-butene", processed_data)
filter!(row -> row.species != "cis-2-pentene", processed_data) ;

# processed_data


In [122]:
num_rows = nrow(processed_data)
training_data = DataFrame(
    species = fill(missing, num_rows),
    CAS = fill(missing,num_rows),
    isomeric_SMILES = fill(missing, num_rows), 
    family = fill(missing, num_rows),
    Mw = fill(missing, num_rows),
    interaction = fill(missing,num_rows),
    source = fill(missing, num_rows),
    critical_temperature_K = fill(0,num_rows),
    critical_pressure_MPa = fill(0,num_rows),
    critical_volume_m3_per_mol = fill(0,num_rows),
    sat_temperatures_K = fill([0.0],num_rows),
    sat_pressures_MPa = fill([0.0],num_rows),
    sat_volumes_liq_m3_per_mol = fill([0.0],num_rows),
    sat_volumes_vap_m3_per_mol = fill([0.0],num_rows),
    
    p_sat_AAD = fill(0.0,num_rows),
    p_sat_AAD_outliers = fill(0.0,num_rows),
    rho_liq_AAD = fill(0.0,num_rows),
    rho_liq_sat_AAD = fill(0.0,num_rows),
    rho_liq_sp_AAD = fill(0.0,num_rows),
    rho_vap_AAD_outliers = fill(0.0,num_rows),

    p_sat_n_points = fill(0,num_rows),
    p_sat_n_points_outliers = fill(0,num_rows),
    rho_liq_n_points = fill(0,num_rows),
    rho_liq_sat_n_points = fill(0,num_rows),
    rho_liq_sp_n_points = fill(0,num_rows),
    rho_vap_n_points_outliers = fill(0,num_rows),
) ;

training_data.species = processed_data.species
training_data.CAS = processed_data.CAS
training_data.isomeric_SMILES = processed_data.isomeric_SMILES
training_data.family = processed_data.family
training_data.Mw = processed_data.Mw
training_data.interaction = processed_data.interaction
training_data.source = processed_data.source ;

training_data.critical_temperature_K = Float64.(training_data.critical_temperature_K)
training_data.critical_pressure_MPa = Float64.(training_data.critical_pressure_MPa)
training_data.critical_volume_m3_per_mol = Float64.(training_data.critical_volume_m3_per_mol) ;

In [123]:
# Check number of functional groups in dataset & number of species per fucntional group
unique_values = unique(processed_data[!, :family])
num_unique_values = length(unique_values)

dataset_summary = DataFrame(
    functional_group = unique_values,
    occurances = fill(0,num_unique_values)
)

for i = 1:length(unique_values)
     count_occurrences = count(x -> x == unique_values[i], processed_data.family)
     dataset_summary[i,:occurances] = count_occurrences ;
end

# dataset_summary

In [124]:
# This cell generates data for all ~1800 species in Esper et al.
num_rows = nrow(processed_data)

n = 500

for i in species_names

    row_number = findall(processed_data.species .== i)[1]
    println(row_number," ",i)

    model = PPCSAFT(i) ;
    critical_props = crit_pure(model)

    temp_range = collect(range(0.5*critical_props[1],critical_props[1],n)) ;
    sat_props = [saturation_pressure(model,temp_range[i]) for i ∈ 1:n] ;

    sat_pressures = [sat_props[i][1] for i ∈ 1:n] ;
    sat_vols_liq = [sat_props[i][2] for i ∈ 1:n] ;
    sat_vols_vap = [sat_props[i][3] for i ∈ 1:n] ;

    for i in 1:length(critical_props)
        training_data[row_number, 7 + i] = critical_props[i]
    end

    training_data[row_number,:sat_temperatures_K] = temp_range
    training_data[row_number,:sat_pressures_MPa] = sat_pressures
    training_data[row_number,:sat_volumes_liq_m3_per_mol] = sat_vols_liq
    training_data[row_number,:sat_volumes_vap_m3_per_mol] = sat_vols_vap

end


1 methane
2 ammonia
3 ammonia-d3
4 water
5 tritium oxide
6 hydrogen fluoride
7 ethyne
8 hydrogen cyanide
9 carbon monoxide
10 nitrogen
11 ethylene
12 ethylene-d4
13 nitrogen oxide
14 formaldehyde
15 ethane
16 methylamine
17 oxygen
18 silane
19 methanol
20 o-deuteromethanol
21 hydrazine
22 hydrogen sulfide
23 deuterium sulfide
24 phosphorous trihydride
25 hydrogen peroxide
26 methyl fluoride
27 hydrogen chloride
28 fluorine
29 argon
30 propyne
31 propadiene
32 acetonitrile
33 propylene
34 cyclopropane
35 ethylenimine
36 carbon dioxide
37 dinitrogen monoxide
38 acetaldehyde
39 ethylene oxide
40 propane
41 formamide
42 dimethylamine
43 ethylamine
44 nitrogen dioxide
45 formic acid
46 ethanol
47 dimethyl ether
48 methylhydrazine
49 ozone
50 methanethiol
51 fluoroethane
52 methyl chloride
53 cyanogen
54 difluoromethane
55 1-buten-3-yne
56 acrylonitrile
57 1,3-butadiene
58 2-butyne
59 1-butyne
60 propionitrile
61 acrolein
62 2-propyn-1-ol
63 1-butene
64 trans-2-butene
65 isobutylene
66 cyclo

In [125]:
# # All phase envelopes plot
# colour_grad_1 = collect(range(1,194,length(species_names)) ./255) 
# colour_grad_2 = collect(range(194,1,length(species_names)) ./255)

# plt.figure(figsize=(8, 6), dpi=400)

# for i = 1:length(species_names)

# plt.semilogx(1e-3 ./training_data[i,12],training_data[i,10], color = [colour_grad_1[i] 0 colour_grad_2[i]])
# plt.semilogx(1e-3 ./training_data[i,13],training_data[i,10],color = [colour_grad_1[i] 0 colour_grad_2[i]])

# end

# # plt.legend(loc="lower right",bbox_to_anchor =(-0.1,-0.05),frameon=false,fontsize=8,ncol=3) 
# plt.xlabel("Density / (mol/L)",fontsize=12)
# plt.ylabel("Temperature / K",fontsize=12)

# plt.savefig("all_compound_phase_envelopes.png",dpi=400)


In [130]:
# Add statistics to training data dataframe
for i = 1:nrow(training_data)
    
    cas_lookup = training_data[i,:CAS]
    index_in_processed_data_statistics = findfirst(processed_data_statistics.CAS .== cas_lookup)

    training_data[i,:p_sat_AAD] = processed_data_statistics[index_in_processed_data_statistics,:p_sat_AAD]
    training_data[i,:p_sat_AAD_outliers] = processed_data_statistics[index_in_processed_data_statistics,:p_sat_AAD_outliers]
    training_data[i,:rho_liq_AAD] = processed_data_statistics[index_in_processed_data_statistics,:rho_liq_AAD]
    training_data[i,:rho_liq_sat_AAD] = processed_data_statistics[index_in_processed_data_statistics,:rho_liq_sat_AAD]
    training_data[i,:rho_liq_sp_AAD] = processed_data_statistics[index_in_processed_data_statistics,:rho_liq_sp_AAD]
    training_data[i,:rho_vap_AAD_outliers] = processed_data_statistics[index_in_processed_data_statistics,:rho_vap_AAD_outliers]

    training_data[i,:p_sat_n_points] = processed_data_statistics[index_in_processed_data_statistics,:p_sat_n_points]
    training_data[i,:p_sat_n_points_outliers] = processed_data_statistics[index_in_processed_data_statistics,:p_sat_n_points_outliers]
    training_data[i,:rho_liq_n_points] = processed_data_statistics[index_in_processed_data_statistics,:rho_liq_n_points]
    training_data[i,:rho_liq_sat_n_points] = processed_data_statistics[index_in_processed_data_statistics,:rho_liq_sat_n_points]
    training_data[i,:rho_liq_sp_n_points] = processed_data_statistics[index_in_processed_data_statistics,:rho_liq_sp_n_points]
    training_data[i,:rho_vap_n_points_outliers] = processed_data_statistics[index_in_processed_data_statistics,:rho_vap_n_points_outliers]

end


In [127]:
# Data storage
CSV.write("training_data.csv", training_data) ;

In [129]:
training_data = CSV.read("training_data.csv",DataFrame)
num_rows = nrow(training_data)
rename!(training_data,:sat_temperatures_K=>:sat_temp_strings, :sat_pressures_MPa=>:sat_pressure_strings,
        :sat_volumes_liq_m3_per_mol=>:sat_vol_liq_strings,:sat_volumes_vap_m3_per_mol=>:sat_vol_vap_strings)

# Define the column names
array_cols_to_create = [:sat_temperatures_K, :sat_pressures_MPa, :sat_volumes_liq_m3_per_mol, :sat_volumes_vap_m3_per_mol]
string_columns = [:sat_temp_strings, :sat_pressure_strings, :sat_vol_liq_strings, :sat_vol_vap_strings]

# Initialize the columns with zeros
for col in array_cols_to_create
    training_data[!, col] .= fill([0.0], num_rows)
end

# Process and assign the values to the DataFrame
for (col, str_col) in zip(array_cols_to_create, string_columns)
    for i = 1:num_rows
        string_sq_brackets_removed = replace(training_data[i, str_col], r"[][]" => "")
        vector_of_floats = [parse(Float64, val) for val in split(string_sq_brackets_removed, ",")]
        training_data[i, col] = vector_of_floats
    end
end

# Remove the string columns
select!(training_data, Not(string_columns...)) ;

training_data

Row,species,CAS,isomeric_SMILES,family,Mw,interaction,source,critical_temperature_K,critical_pressure_MPa,critical_volume_m3_per_mol,p_sat_AAD,p_sat_AAD_outliers,rho_liq_AAD,rho_liq_sat_AAD,rho_liq_sp_AAD,rho_vap_AAD_outliers,p_sat_n_points,p_sat_n_points_outliers,rho_liq_n_points,rho_liq_sat_n_points,rho_liq_sp_n_points,rho_vap_n_points_outliers,sat_temperatures_K,sat_pressures_MPa,sat_volumes_liq_m3_per_mol,sat_volumes_vap_m3_per_mol
,String,String15,String,String31,Float64,String15,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Array…,Array…,Array…,Array…
1,methane,74-82-8,C,Alkanes,16.031,NONPOLAR,10.1021/acs.iecr.3c02255,191.453,4.68922e6,0.000108063,0.59,0.72,0.53,0.48,0.68,2.65,1211,1222,1126,856,270,4317,"[95.7267, 95.9186, 96.1104, 96.3023, 96.4941, 96.6859, 96.8778, 97.0696, 97.2614, 97.4533 … 189.727, 189.919, 190.111, 190.302, 190.494, 190.686, 190.878, 191.07, 191.262, 191.453]","[21706.6, 22184.2, 22670.1, 23164.7, 23667.8, 24179.7, 24700.4, 25230.1, 25768.8, 26316.7 … 4.46102e6, 4.48608e6, 4.51122e6, 4.53643e6, 4.56172e6, 4.58707e6, 4.6125e6, 4.63801e6, 4.66358e6, 4.68922e6]","[3.60443e-5, 3.60648e-5, 3.60854e-5, 3.61059e-5, 3.61265e-5, 3.61471e-5, 3.61678e-5, 3.61885e-5, 3.62092e-5, 3.62299e-5 … 8.20978e-5, 8.33085e-5, 8.46313e-5, 8.60919e-5, 8.77271e-5, 8.95939e-5, 9.17882e-5, 9.44977e-5, 9.82083e-5, 0.000108063]","[0.036283, 0.0355668, 0.0348676, 0.0341852, 0.0335189, 0.0328684, 0.0322333, 0.031613, 0.0310073, 0.0304156 … 0.00014731, 0.000144572, 0.000141735, 0.000138772, 0.000135648, 0.000132305, 0.000128648, 0.000124489, 0.000119342, 0.000108063]"
2,ammonia,7664-41-7,N,Inorganic compounds,17.027,ASSOCIATIVE,10.1021/acs.iecr.3c02255,423.884,1.50571e7,7.2694e-5,1.23,1.4,1.42,1.38,1.7,10.78,1082,1093,879,775,104,180,"[211.942, 212.367, 212.792, 213.216, 213.641, 214.066, 214.491, 214.915, 215.34, 215.765 … 420.062, 420.486, 420.911, 421.336, 421.761, 422.185, 422.61, 423.035, 423.46, 423.884]","[19918.2, 20497.6, 21091.0, 21698.6, 22320.8, 22957.8, 23609.9, 24277.4, 24960.5, 25659.5 … 1.4172e7, 1.42682e7, 1.43649e7, 1.44622e7, 1.456e7, 1.46583e7, 1.47572e7, 1.48566e7, 1.49566e7, 1.50571e7]","[2.43847e-5, 2.44004e-5, 2.44161e-5, 2.44319e-5, 2.44477e-5, 2.44635e-5, 2.44794e-5, 2.44952e-5, 2.45111e-5, 2.4527e-5 … 5.57947e-5, 5.65501e-5, 5.73779e-5, 5.82953e-5, 5.9327e-5, 6.05115e-5, 6.19137e-5, 6.36609e-5, 6.60837e-5, 7.2694e-5]","[0.0881662, 0.0858388, 0.0835837, 0.0813982, 0.07928, 0.0772267, 0.075236, 0.0733058, 0.0714339, 0.0696184 … 0.000101229, 9.91459e-5, 9.69995e-5, 9.47723e-5, 9.24394e-5, 8.99624e-5, 8.72763e-5, 8.42535e-5, 8.05634e-5, 7.2694e-5]"
3,ammonia-d3,13550-49-7,[2H]N([2H])[2H],Other compounds,17.027,ASSOCIATIVE,10.1021/acs.iecr.3c02255,419.98,1.47831e7,6.85203e-5,0.27,0.31,0.09,0.13,0.04,-1.0,231,232,20,12,8,-1,"[209.99, 210.411, 210.832, 211.253, 211.673, 212.094, 212.515, 212.936, 213.357, 213.777 … 416.193, 416.614, 417.034, 417.455, 417.876, 418.297, 418.718, 419.138, 419.559, 419.98]","[14942.5, 15384.8, 15838.2, 16302.8, 16778.9, 17266.7, 17766.5, 18278.4, 18802.7, 19339.6 … 1.39215e7, 1.40154e7, 1.41099e7, 1.42047e7, 1.43e7, 1.43957e7, 1.44919e7, 1.45885e7, 1.46856e7, 1.47831e7]","[1.9988e-5, 2.00015e-5, 2.00151e-5, 2.00286e-5, 2.00422e-5, 2.00558e-5, 2.00695e-5, 2.00831e-5, 2.00968e-5, 2.01105e-5 … 5.25051e-5, 5.32469e-5, 5.40563e-5, 5.49491e-5, 5.59483e-5, 5.70892e-5, 5.84318e-5, 6.00934e-5, 6.23784e-5, 6.85203e-5]","[0.11499, 0.111879, 0.108866, 0.105947, 0.103118, 0.100377, 0.0977211, 0.0951465, 0.0926506, 0.0902308 … 9.4244e-5, 9.23853e-5, 9.04688e-5, 8.84785e-5, 8.63914e-5, 8.4172e-5, 8.17604e-5, 7.90392e-5, 7.57036e-5, 6.85203e-5]"
4,water,7732-18-5,O,Inorganic compounds,18.011,ASSOCIATIVE,10.1021/acs.iecr.3c02255,677.938,3.02134e7,5.80502e-5,1.1,1.18,2.21,2.23,0.76,7.52,5870,5896,6342,6244,98,1827,"[338.969, 339.648, 340.327, 341.007, 341.686, 342.365, 343.045, 343.724, 344.403, 345.082 … 671.824, 672.503, 673.